In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -U tensorflow-addons~=0.7.0
!pip install -U tensorflow
!pip install -U keras

     |████████████████████████████████| 993kB 4.7MB/s 
     |████████████████████████████████| 421.8MB 16kB/s 
     |████████████████████████████████| 3.9MB 36.5MB/s 
     |████████████████████████████████| 450kB 38.1MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.1.0)
     |████████████████████████████████| 378kB 4.7MB/s 
  Found existing installation: Keras 2.2.5

In [0]:
# only colab code

In [0]:
# 압축파일 옮기기
!mkdir -p /content/kaggle/working/dataset
!cp /content/drive/'My Drive'/kaggle/working/dataset/* /content/kaggle/working/dataset/

In [0]:
# metadata.json 옮기기
!mkdir -p /content/kaggle/input/deepfake-detection-challenge/train_sample_videos
!cp -r /content/drive/'My Drive'/kaggle/input/deepfake-detection-challenge/train_sample_videos/metadata.json /content/kaggle/input/deepfake-detection-challenge/train_sample_videos/

In [0]:
!unzip -q /content/kaggle/working/dataset/REAL3.zip -d /content/kaggle/working/dataset/REAL2
!unzip -q /content/kaggle/working/dataset/FAKE3.zip -d /content/kaggle/working/dataset/FAKE2

In [0]:
import os
os.getcwd()
os.chdir('/content/kaggle/working')

In [0]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [0]:
train_sample_metadata = pd.read_json('../input/deepfake-detection-challenge/train_sample_videos/metadata.json').T
train_sample_metadata

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4
...,...,...,...
etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4
etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4
eudeqjhdfd.mp4,REAL,train,None


In [0]:
# 전처리한 이미지 불러오기
input_shape = (160, 160, 3)
data_dir = '../working/dataset'

real_data = [f for f in os.listdir(data_dir+'/REAL2') if f.endswith('.jpg')]
fake_data = [f for f in os.listdir(data_dir+'/FAKE2') if f.endswith('.jpg')]

X = []
Y = []

# 이미지 불러서 list로 저장
for idx, img in enumerate(fake_data):
    X.append(img_to_array(load_img(data_dir+'/FAKE2/'+img)).flatten() / 255.0)
    Y.append(0)
    original_img = img.split('_')[-1].split('.')[0]
    frame = img.split('_')[2]
    X.append(img_to_array(load_img(data_dir+f'/REAL2/REAL_{original_img}_{frame}_0.jpg' )).flatten() / 255.0)
    Y.append(1)
    print('*',end='')

    if (idx+1) % 500 == 0:
        print(idx+1)

Y_val_org = Y

#Normalization
X = np.array(X)
Y = to_categorical(Y, 2)

#Reshape
X = X.reshape(-1, 160, 160, 3)

#Train-Test split
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = 0.1)

#X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.2)
print(len(X_train)*8//10)

split_eight = len(X_train)*8//10

********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************500
****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

In [0]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


googleNet_model = InceptionResNetV2(include_top=False, input_shape=input_shape) #  weights='imagenet',
googleNet_model.trainable = True

x = googleNet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2, activation='softmax', name='softmax')(x)
model = Model(googleNet_model.input, x)

# model = Sequential()
# model.add(googleNet_model)
# model.add(GlobalAveragePooling2D())
# model.add(Dense(units=2, activation='softmax'))

# model.compile(loss='binary_crossentropy',
#               optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
#               metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 79, 79, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 79, 79, 32)   96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 79, 79, 32)   0           batch_normalization_203[0][0]    
____________________________________________________________________________________________

In [0]:
# #efficientnet
# import efficientnet.keras as efn

# model = efn.EfficientNetB0(weights='imagenet')  # or weights='noisy-student'
# model.summary()

In [0]:
# #Xception
# model = Xception(#include_top=False,
#                  weights=None,
#                  input_shape=(height, width, 3),classes = 2)

# #model = multi_gpu_model(model, gpus=2)

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.summary()

In [0]:
split_eight

22025

In [0]:
early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0,
                               patience=2,
                               verbose=0, mode='auto')
#mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# adam = optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

EPOCHS = 100
BATCH_SIZE = 128
history = model.fit(X_train[:split_eight], Y_train[:split_eight], batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (X_train[split_eight:], Y_train[split_eight:]), verbose = 1,  callbacks=[early_stopping]) #, mc

Train on 22025 samples, validate on 5507 samples
Epoch 1/100
22025/22025 [==============================] - 137s 6ms/sample - loss: 0.3697 - accuracy: 0.8296 - val_loss: 0.4205 - val_accuracy: 0.8329
Epoch 2/100
22025/22025 [==============================] - 103s 5ms/sample - loss: 0.0852 - accuracy: 0.9705 - val_loss: 0.1758 - val_accuracy: 0.9463
Epoch 3/100
22025/22025 [==============================] - 103s 5ms/sample - loss: 0.0299 - accuracy: 0.9913 - val_loss: 0.0648 - val_accuracy: 0.9748
Epoch 4/100
22025/22025 [==============================] - 103s 5ms/sample - loss: 0.0156 - accuracy: 0.9958 - val_loss: 0.0522 - val_accuracy: 0.9804
Epoch 5/100
22025/22025 [==============================] - 103s 5ms/sample - loss: 0.0100 - accuracy: 0.9975 - val_loss: 0.0575 - val_accuracy: 0.9824
Epoch 6/100
22025/22025 [==============================] - 103s 5ms/sample - loss: 0.0087 - accuracy: 0.9977 - val_loss: 0.0613 - val_accuracy: 0.9829


In [0]:
model.evaluate(X_validation, Y_validation)

3060/3060 [==============================] - 8s 3ms/sample - loss: 0.0799 - accuracy: 0.9820


[0.07986413054959544, 0.98202616]

In [0]:
model.save('deepfake-detection2.h5')

In [0]:
!cp /content/kaggle/working/deepfake-detection-model-xception2.h5 /content/drive/'My Drive'/kaggle/